**<center><font face="Times New Roman" size=6> Implied Volatility <center>**

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Newton-Method" data-toc-modified-id="Newton-Method-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Newton Method</a></span></li><li><span><a href="#Bisection-Method" data-toc-modified-id="Bisection-Method-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Bisection Method</a></span></li><li><span><a href="#SPY-Option" data-toc-modified-id="SPY-Option-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>SPY Option</a></span></li></ul></div>

In [1]:
# Data Manipulation
import pandas as pd
import numpy as np

# Import BlackScholes
from BS_class import BS

# Plotting
import cufflinks as cf
cf.set_config_file(offline=True)

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Newton Method

In [2]:
def newton_iv(className, spot, strike, dte, rate, volatility, dividends, callprice=None, putprice=None):
    
    x0 = 1
    h = 0.001
    tolerance = 1e-7
    epsilon = 1e-14                             
    
    maxiter = 300
    
    if callprice:
        f = lambda x: eval(className)(spot, strike, dte, rate, x, dividends).callPrice - callprice
    if putprice:
        f = lambda x: eval(className)(spot, strike, dte, rate, x, dividends).putPrice - putprice
        
    for i in range(maxiter):
        y = f(x0)                               
        yprime = (f(x0+h) - f(x0-h))/(2*h)      
        
        if abs(yprime)<epsilon:
            break                               
        x1 = x0 - y/yprime
        
        if (abs(x1-x0) <= tolerance*abs(x1)):
            break
            
        x0=x1
        
    return x1

# Bisection Method

In [3]:
def bisection_iv(className, spot, strike, dte, rate, volatility, dividends, callprice=None, putprice=None):
    
    high=500.0
    low=0.0
    
    # this is market price
    if callprice:
        price = callprice
    if putprice and not callprice:
        price = putprice
        
    tolerance = 1e-7
        
    for i in range(1000):
        mid = (high + low) / 2              
        if mid < tolerance:
            mid = tolerance
            
        if callprice:
            estimate = eval(className)(spot, strike, dte, rate, mid, dividends).callPrice 
        if putprice:
            estimate = eval(className)(spot, strike, dte, rate, mid, dividends).putPrice
        
        if round(estimate,6) == price:
            break
        elif estimate > price: 
            high = mid                     
        elif estimate < price: 
            low = mid                       
    
    return mid

# SPY Option

In [4]:
opt_data = pd.read_csv('/Users/antoneyoung/Jupyter Notebook/OptionMetrics/Final_Data_2.csv')

In [5]:
opt_date = opt_data.set_index(['Date'])

In [6]:
ivfunction_test = opt_date.loc['2021-12-31'].query('Maturity == 364')[:6]

In [7]:
ivfunction_test

,SpotPrice,Expiration,Maturity,ForwardPrice,Strike,Dividend,CallBid,CallOffer,CallMid,IV_Call,PutBid,PutOffer,PutMid,IV_Put,RiskFreeRate
Date,,,,,,,,,,,,,,,
2021-12-31,4766.18,2022-12-30,364,4739.960187,1600,0.012432,3122.9,3146.9,3134.9,0.570805,1.0,20.2,10.6,-99.990000,0.006901
2021-12-31,4766.18,2022-12-30,364,4739.960187,1700,0.012432,3025.3,3049.3,3037.3,0.550412,2.7,21.9,12.3,0.515698,0.006901
2021-12-31,4766.18,2022-12-30,364,4739.960187,1800,0.012432,2927.8,2951.8,2939.8,0.530916,4.5,23.7,14.1,0.499953,0.006901
2021-12-31,4766.18,2022-12-30,364,4739.960187,1900,0.012432,2830.5,2854.5,2842.5,0.512622,5.0,31.0,18.0,0.494393,0.006901
2021-12-31,4766.18,2022-12-30,364,4739.960187,2000,0.012432,2733.4,2757.4,2745.4,0.495308,5.0,33.0,19.0,0.473615,0.006901
2021-12-31,4766.18,2022-12-30,364,4739.960187,2100,0.012432,2636.5,2660.5,2648.5,0.478793,5.5,35.5,20.5,0.455611,0.006901


In [8]:
i = 0
S0 = ivfunction_test.SpotPrice.iloc[i]
K = ivfunction_test.Strike.iloc[i]
T = ivfunction_test.Maturity.iloc[i]/365
r = ivfunction_test.RiskFreeRate.iloc[i]
q = ivfunction_test.Dividend.iloc[i]
callprice = ivfunction_test.CallMid.iloc[i]
impvol_newton = newton_iv('BS', S0, K, T, r, 0.2, q, callprice)
impvol_bisection = bisection_iv('BS', S0, K, T, r, 0.2, q, callprice)

In [9]:
print('====================================')
print('Newton Method   :', impvol_newton)
print('====================================')
print('Bisection Method:', impvol_bisection)
print('====================================')

Newton Method   : 0.5709375328687041
Bisection Method: 0.5709375327569433


Reference:

The Model 3 Python Lab notes in CQF